In [36]:
!pip install flask_pymongo

In [16]:
#Importing dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import pymongo
import requests
import time
# from IPython.display import Image
# import shutil


In [6]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\samue\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [7]:
#going to nasa new site to scape title and paragrapgh
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html

#sleep broswer for 1 sec to give time to collect html
time.sleep(1)

soup = bs(html, 'lxml')
# print(soup.prettify())

In [8]:
#selecting slide item  to later use to extract news title and paragraph
slide_elem = soup.select_one("ul.item_list li.slide")


In [9]:
# get the title and paragraph that went with it, then print

tl = slide_elem.find("div", class_="content_title").get_text()
p = slide_elem.find("div", class_="article_teaser_body").get_text()

print(tl)
print(p)

Mars Is Getting a New Robotic Meteorologist
Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.


In [10]:
# visiting site to collect image for articles
url = "https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA22109"
browser.visit(url)
html = browser.html
soup = bs(html, "lxml")

In [20]:
# extract img url from site
image = soup.find_all("div", class_="img")[0].img["src"]
f_image_url = f'https://www.jpl.nasa.gov{image}'

print(f_image_url)

In [10]:
# extract the mars fact
url = "https://space-facts.com/mars/"
table = pd.read_html(url)

#putting into data frame and transposing 
table = table[0].set_index(0).T

# showing results in data frame
table

,Equatorial Diameter:,Polar Diameter:,Mass:,Moons:,Orbit Distance:,Orbit Period:,Surface Temperature:,First Record:,Recorded By:
1,"6,792 km","6,752 km",6.39 × 10^23 kg (0.11 Earths),2 (Phobos & Deimos),"227,943,824 km (1.38 AU)",687 days (1.9 years),-87 to -5 °C,2nd millennium BC,Egyptian astronomers


In [11]:
# turning data frame into html table 
mars_info = table.to_html()
print(mars_info)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Equatorial Diameter:</th>
      <th>Polar Diameter:</th>
      <th>Mass:</th>
      <th>Moons:</th>
      <th>Orbit Distance:</th>
      <th>Orbit Period:</th>
      <th>Surface Temperature:</th>
      <th>First Record:</th>
      <th>Recorded By:</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>6,792 km</td>
      <td>6,752 km</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
      <td>2 (Phobos &amp; Deimos)</td>
      <td>227,943,824 km (1.38 AU)</td>
      <td>687 days (1.9 years)</td>
      <td>-87 to -5 °C</td>
      <td>2nd millennium BC</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [12]:
#using web driver to visit website to get hemisphere images and titles
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = bs(html, "lxml")

In [13]:

# loop through the links and add the urls
h_imgs = []

ht = soup.find_all("h3")

for t in ht:
    
    #creating dict to hold titles and urls 
    h_dict = {}
    title = t.text.strip()
    browser.click_link_by_partial_text(title)
    
    #adding to dict and making a list of dicts
    h_dict["title"] = title
    h_dict["img_url"] = browser.find_by_text("Sample")["href"]
    h_imgs.append(h_dict)
    
    # return to web page
    browser.visit(url)
    
h_imgs



C:\Users\samue\Miniconda3\envs\pythondata\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [14]:
mars_data = {}

mars_data["tl"] = tl
mars_data["p"] = p
mars_data["f_image_url"] = f_image_url
mars_data["mars_info"] = mars_info
mars_data["h_imgs"] = h_imgs

mars_data

{'tl': 'Mars Is Getting a New Robotic Meteorologist',
 'p': "Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.",
 'f_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA22110-640x350.jpg',
 'mars_info': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Equatorial Diameter:</th>\n      <th>Polar Diameter:</th>\n      <th>Mass:</th>\n      <th>Moons:</th>\n      <th>Orbit Distance:</th>\n      <th>Orbit Period:</th>\n      <th>Surface Temperature:</th>\n      <th>First Record:</th>\n      <th>Recorded By:</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>1</th>\n      <td>6,792 km</td>\n      <td>6,752 km</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n      <td>227,943,824 km (1.38 AU)</td>\n      <td>687 days (1.9 years)</td>\n      <td>-87 to -5 °C</td>\n      <td>